<a href="https://colab.research.google.com/github/m-cheicki/ESILV_Neural_Networks/blob/main/CNN_for_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network for CIFAR-10 - Image Classification

## How to activate GPU in Google Colab